In [1]:
import torch
import platform
import pathlib

if platform.system().lower() == 'windows':
    temp = pathlib.PosixPath
    pathlib.PosixPath = pathlib.WindowsPath
else:
    temp = pathlib.WindowsPath
    pathlib.WindowsPath = pathlib.PosixPath

In [2]:
checkpoint = torch.load('./checkpoints/vallex-checkpoint.pt', map_location='cpu')

/var/folders/fd/vflcczp10w97j9zp71gwynl40000gn/T/ipykernel_3391/3491729396.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('./checkpoints/vallex-

language_embedding 的权重是一个语言数量 x 1024 的矩阵，我们要新增一个语言，因此扩展一个 1 x 1024 的矩阵，并随机初始化。

In [3]:
ar_lang_emb = checkpoint['model']['ar_language_embedding.word_embeddings.weight']
checkpoint['model']['ar_language_embedding.word_embeddings.weight'] = torch.cat([ar_lang_emb, torch.rand(1, 1024)])

nar_lang_emb = checkpoint['model']['nar_language_embedding.word_embeddings.weight']
checkpoint['model']['nar_language_embedding.word_embeddings.weight'] = torch.cat([nar_lang_emb, torch.rand(1, 1024)])

保存魔改后的模型权重。

In [4]:
torch.save(checkpoint, './checkpoints/vallex-checkpoint2.pt')

验证新模型可以加载新权重。

In [5]:
from macros import *
from models.vallex import VALLE

checkpoint2 = torch.load('./checkpoints/vallex-checkpoint2.pt', map_location='cpu')
model = VALLE(
        N_DIM,
        NUM_HEAD,
        NUM_LAYERS,
        norm_first=True,
        add_prenet=False,
        prefix_mode=PREFIX_MODE,
        share_embedding=True,
        nar_scale_factor=1.0,
        prepend_bos=True,
        num_quantizers=NUM_QUANTIZERS,
    )
missing_keys, unexpected_keys = model.load_state_dict(
    checkpoint2["model"], strict=True
)
assert not missing_keys
model.eval()

/var/folders/fd/vflcczp10w97j9zp71gwynl40000gn/T/ipykernel_3391/2014426536.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint2 = torch.load('./checkpoints/vallex

VALLE(
  (ar_text_embedding): TokenEmbedding(
    (dropout): Dropout(p=0.0, inplace=False)
    (word_embeddings): Embedding(2048, 1024)
  )
  (nar_text_embedding): TokenEmbedding(
    (dropout): Dropout(p=0.0, inplace=False)
    (word_embeddings): Embedding(2048, 1024)
  )
  (ar_audio_embedding): TokenEmbedding(
    (dropout): Dropout(p=0.0, inplace=False)
    (word_embeddings): Embedding(1026, 1024)
  )
  (ar_text_prenet): Identity()
  (ar_audio_prenet): Identity()
  (ar_text_position): SinePositionalEmbedding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (ar_audio_position): SinePositionalEmbedding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (ar_decoder): TransformerEncoder(
    (layers): ModuleList(
      (0-11): 12 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)
        )
        (linear1): Linear(in_features=1024, out_features=4096, bias=True)
   

验证新模型无法加载旧权重。

In [6]:
from macros import *
from models.vallex import VALLE

checkpoint = torch.load('./checkpoints/vallex-checkpoint.pt', map_location='cpu')
model = VALLE(
        N_DIM,
        NUM_HEAD,
        NUM_LAYERS,
        norm_first=True,
        add_prenet=False,
        prefix_mode=PREFIX_MODE,
        share_embedding=True,
        nar_scale_factor=1.0,
        prepend_bos=True,
        num_quantizers=NUM_QUANTIZERS,
    )
missing_keys, unexpected_keys = model.load_state_dict(
    checkpoint["model"], strict=True
)
assert not missing_keys
model.eval()

/var/folders/fd/vflcczp10w97j9zp71gwynl40000gn/T/ipykernel_3391/971458241.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('./checkpoints/vallex-c

RuntimeError: Error(s) in loading state_dict for VALLE:
	size mismatch for ar_language_embedding.word_embeddings.weight: copying a param with shape torch.Size([3, 1024]) from checkpoint, the shape in current model is torch.Size([4, 1024]).
	size mismatch for nar_language_embedding.word_embeddings.weight: copying a param with shape torch.Size([3, 1024]) from checkpoint, the shape in current model is torch.Size([4, 1024]).